# 1. Importing the data and creating the data frame

Importing necessary libraries.

In [ ]:
# 1. Authenticate and set up BigQuery connection
!pip install --upgrade google-cloud-bigquery
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import pandas as pd
import re

from google.colab import drive

Bringing the table back from BigQuery

In [ ]:
# 2. Set project and table identifiers
project_id = "musicalanalysisproject"
dataset_id = "spotify_data"
table_id = "spotify_dataset"

client = bigquery.Client(project=project_id)


# 3. Query to extract all fields from the table in BigQuery
query = f"""
SELECT
    *
FROM `{project_id}.{dataset_id}.{table_id}`
WHERE text IS NOT NULL
ORDER BY Release_Date
"""

The column "Release_Date" in the data frame needs some adjustments that are being made in the code below.

In [ ]:
# Create the df
df = client.query(query).to_dataframe()

# Clean suffixes like "1st", "2nd", "3rd", etc.
df['Release_Date_clean'] = df['Release_Date'].str.replace(
    r'(\d{1,2})(st|nd|rd|th)', r'\1', regex=True)

# Convert to datetime
df['Release_Date'] = pd.to_datetime(df['Release_Date_clean'], errors='coerce', dayfirst=True)

# Drop auxiliary column
df.drop(columns=['Release_Date_clean'], inplace=True)

# Create extra columns
df['year'] = df['Release_Date'].dt.year
df['month'] = df['Release_Date'].dt.month
df['day'] = df['Release_Date'].dt.day
df['decade'] = (df['year']//10) * 10

# Filter the years
df = df[df['year'].between(1990, 2024)]

# Reset index (bc BigQuery doesn't have order, but the ORDER BY from previous code created some order that needs to be fixed)
df = df.reset_index(drop=True)

In [ ]:
# Renaming the column 'song' for better reference
df.rename(columns = {'song':'title'})

,Artists,title,text,Length,emotion,Genre,Album,Release_Date,Key,Tempo,...,Similar_Artist_2,Similar_Song_2,Similarity_Score_2,Similar_Artist_3,Similar_Song_3,Similarity_Score_3,year,month,day,decade
0,"Public Enemy,Ice Cube,Big Daddy Kane",Burn Hollywood Burn,[Chuck D] Burn Hollywood burn I smell a riot G...,02:47,anger,hip hop,Fear Of A Black Planet,1990-04-10,D Maj,116,...,"FEVER 333,Yelawolf",The First Stone Changes,0.969175,"Torren Foot,Tinie Tempah,L Devine",Naked Alone,0.966809,1990,4,10,1990
1,Public Enemy,Incident at 66.6 FM,"Flavor Flav: Yeah! It's the Flav, holmes, cold...",01:37,anger,"rap,hip-hop,hip hop",Fear Of A Black Planet,1990-04-10,F# Maj,113,...,Beastie Boys,Brrr Stick Em,0.943620,"Rizván,N8TXU,LNGSLV",Living like a G,0.942875,1990,4,10,1990
2,Public Enemy,Brothers Gonna Work It Out,"[Intro] “The rhythm, the rebel” “Don’t believe...",05:07,joy,"rap,hip-hop,hip hop",Fear Of A Black Planet,1990-04-10,G Maj,105,...,Intro,Love Thang,0.961616,Django Django,First Light,0.958451,1990,4,10,1990
3,Public Enemy,Cant Do Nuttin For Ya Man,"[Verse 1] Runnin' for your life, by the knife ...",02:46,anger,"rap,hip-hop,hip hop",Fear Of A Black Planet,1990-04-10,E min,118,...,"Plies,DaBaby",Boss Friends,0.989446,"DaBaby,Kodak Black",LEVELS,0.988179,1990,4,10,1990
4,Fleetwood Mac,In The Back Of My Mind,Well I try and I try You can't forget her ...,07:02,anger,"pop rock,classic rock,psychedelic rock",Behind the Mask,1990-04-10,D Maj,157,...,IAMX,Oh Beautiful Town,0.953364,HIM,The Path,0.935916,1990,4,10,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502189,Kenneth Ewards,Cold Blooded,[Intro] What it do Flight Crew FTC Flight Team...,03:12,fear,soul,Flight Flow Vol 6,2023-09-09,D# min,130,...,Steven Barrera,Will You Lie,0.976467,Summer Time Chillout Music Ensemble & Chillout...,Be More Chill - Part 2,0.975662,2023,9,9,2020
502190,Panic! At The Disco,IDGAF,[Verse 1] You call me all friendly Tellin' me ...,03:28,joy,"alternative rock,emo,electropop",Indie - Rock & Pop - I.D.G.A.F. - Hits,2023-09-09,B min,110,...,X,Cyrano de Bergers Back,0.986935,Planetshakers,Everything's Changed,0.986463,2023,9,9,2020
502191,"Measoon,Tryhan,Thuglack,Young Molly",Lakers,[Hook: Young Thug] She just wanna get this pap...,02:39,joy,hip hop,Lakers,2023-09-09,F min,82,...,"Pabzzz,Empty Advice",One Way Love,0.954255,"Cal Chuchesta,Evo Auxilium",Coin Star,0.940885,2023,9,9,2020
502192,Ymt Acp,Hard Trap,[Intro] The separation of talent and skill is ...,02:41,joy,hip hop,Hard Living On The Road,2023-09-09,D Maj,109,...,Famous Dex,Read About It Intro,0.972171,Preme,Light Skinned Keisha Speaks,0.971564,2023,9,9,2020


In [ ]:
# View
df.sample(2)

,Artists,song,text,Length,emotion,Genre,Album,Release_Date,Key,Tempo,...,Similar_Artist_2,Similar_Song_2,Similarity_Score_2,Similar_Artist_3,Similar_Song_3,Similarity_Score_3,year,month,day,decade
381389,"SiR,Smino",LA Lisa,"[Verse 1: SiR] No, it ain't a question, not ev...",03:40,anger,hip hop,Chasing Summer,2019-08-30,E min,120,...,Taco Hemingway,4 AM in Girona,0.972413,Bryson Tiller,Losing Focus,0.972124,2019,8,30,2010
404685,Salmonella Dub,We can be Electric,[Verse 1] Please forgive me if this sounds too...,06:46,fear,drum and bass,Inside the Dubplates,2001-08-03,F# min,171,...,"Calibre,DRS",Sunrise,0.985180,"Lenzman,Riya",Open Page,0.980976,2001,8,3,2000


There are also other columns that need to be in a different data type.

In [ ]:
# Convert 'Length' from mm:ss to total seconds and overwrite original column
df["Length"] = pd.to_timedelta("00:" + df["Length"]).dt.total_seconds().astype("Int64")

In [ ]:
df['Length'].head()

,Length
0,167
1,97
2,307
3,166
4,422


In [ ]:
# Convert 'Tempo' to numeric (coerce errors to NaN)
df["Tempo"] = pd.to_numeric(df["Tempo"], errors="coerce")

In [ ]:
# Convert 'Loudness' --> remove 'db' and convert to float
df["Loudness_db"] = df["Loudness_db"].str.replace("db", "", regex=False).astype(float)

In [ ]:
# Converting all boolean columns to boolean datatype
df["Good_for_Party"] = df["Good_for_Party"].astype(bool)
df["Good_for_Work_Study"] = df["Good_for_Work_Study"].astype(bool)
df["Good_for_Relaxation_Meditation"] = df["Good_for_Relaxation_Meditation"].astype(bool)
df["Good_for_Exercise"] = df["Good_for_Exercise"].astype(bool)
df["Good_for_Running"] = df["Good_for_Running"].astype(bool)
df["Good_for_Yoga_Stretching"] = df["Good_for_Yoga_Stretching"].astype(bool)
df["Good_for_Driving"] = df["Good_for_Driving"].astype(bool)
df["Good_for_Social_Gatherings"] = df["Good_for_Social_Gatherings"].astype(bool)
df["Good_for_Morning_Routine"] = df["Good_for_Social_Gatherings"].astype(bool)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502194 entries, 0 to 502193
Data columns (total 43 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Artists                         502194 non-null  object        
 1   song                            502180 non-null  object        
 2   text                            502194 non-null  object        
 3   Length                          502194 non-null  Int64         
 4   emotion                         502194 non-null  object        
 5   Genre                           502194 non-null  object        
 6   Album                           502142 non-null  object        
 7   Release_Date                    502194 non-null  datetime64[ns]
 8   Key                             502194 non-null  object        
 9   Tempo                           502194 non-null  Int64         
 10  Loudness_db                     502194 non-null  float64

# 2. EDA of the data frame

Now that we have created the data frame and all the columns have the correct data type, we will start with the exploratory data analysis (EDA).

In [ ]:
df.describe()

,Length,Release_Date,Tempo,Loudness_db,Popularity,Energy,Danceability,Positiveness,Speechiness,Liveness,Acousticness,Instrumentalness,Similarity_Score_1,Similarity_Score_2,Similarity_Score_3,year,month,day,decade
count,502194.0,502194,502194.0,502194.000000,502194.0,502194.0,502194.0,502194.0,502194.0,502194.0,502194.0,502194.0,502194.000000,502194.000000,502194.000000,502194.000000,502194.000000,502194.000000,502194.000000
mean,222.519192,2014-07-24 02:01:14.925626112,120.513097,-7.786607,31.991864,63.319538,59.720618,46.746554,12.218089,19.663094,24.79774,7.24153,0.984015,0.979027,0.975973,2014.082687,6.291196,14.641171,2009.498441
min,5.0,1990-01-01 00:00:00,31.0,-50.060000,0.0,0.0,6.0,0.0,2.0,1.0,0.0,0.0,0.002656,0.002647,0.002647,1990.000000,1.000000,1.000000,1990.000000
25%,178.0,2010-11-23 06:00:00,96.0,-9.330000,19.0,49.0,48.0,28.0,4.0,10.0,2.0,0.0,0.977381,0.971596,0.967958,2010.000000,3.000000,6.000000,2010.000000
50%,213.0,2017-03-10 00:00:00,120.0,-7.000000,30.0,65.0,61.0,46.0,6.0,13.0,12.0,0.0,0.985826,0.980971,0.978099,2017.000000,6.000000,14.000000,2010.000000
75%,253.0,2020-03-13 00:00:00,140.0,-5.230000,42.0,81.0,73.0,65.0,17.0,25.0,40.0,0.0,0.993348,0.988595,0.986095,2020.000000,9.000000,23.000000,2020.000000
max,3572.0,2024-08-16 00:00:00,200.0,4.980000,100.0,100.0,99.0,100.0,97.0,100.0,100.0,100.0,1.000000,1.000000,1.000000,2024.000000,12.000000,31.000000,2020.000000
std,86.162125,NaN,29.344,3.972802,17.98564,22.070021,17.538568,23.818965,12.62489,16.202996,28.930573,20.844487,0.013581,0.015517,0.016595,8.092569,3.455004,9.313339,8.629157


First, we will remove duplicates.

In [ ]:
df.duplicated().sum()

np.int64(51809)

Around the 10% of the data are duplicates and we will drop them.

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.shape[0]

450385

Now we will investigate why there are songs with such a short length (just some seconds) and some other that are over 1h. Could they maybe be just intros, in the case of the short length songs, or podcast, in the case of long ones?

In [ ]:
df = df.sort_values(by=['Length'])

In [ ]:
df['Length'].isna().value_counts(normalize=True)

,proportion
Length,
False,1.0


After sorting the songs out by "Length", we realize that there are no null values, so all the rows in this column have text (aka, words). This means that these songs are not instrumental intros.

We will define two masks to study the very short songs and the very long ones.

In [ ]:
short_songs = df['Length'] < 30
long_songs = df['Length'] > 3000 # equivalent to 50 minutes

In [ ]:
# Use all the space to show the columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Apply the mask
short_songs_df = df.loc[short_songs, ['text','Length']]
long_songs_df = df.loc[long_songs, ['text','Length']]

With this we realize that there are songs that are impossible to have such long lyrics and very short length, like song 318324 that has 5 seconds length.

Therefore, we will create two new columns in the data set: 'word_count' and 'words_per_sec'. We will consider as outliers the songs with over 10 words per second, and we will remove them from the dataset.

In [ ]:
# Create a copy to work on
short_songs_df = df[df['Length'] < 30].copy()

# Count number of words in the lyrics
short_songs_df['word_count'] = short_songs_df['text'].str.split().apply(len)

# Calculate words per second
short_songs_df['words_per_sec'] = short_songs_df['word_count'] / short_songs_df['Length']

# Inspect songs with suspiciously high values (e.g. above 8 wps)
suspicious_songs = short_songs_df[short_songs_df['words_per_sec'] > 8]

In [ ]:
suspicious_songs[['song','Artists','text','Length','word_count','words_per_sec']].sort_values(by = 'words_per_sec', ascending = False)

song  \
66730                             Freaky Tales   
318324            Light of Day Day of Darkness   
437090                                  Police   
340902                              Philosophy   
378571                             On The Plug   
467784                               Ring Ring   
181199                                Play Off   
88404                          Look Like Money   
67143                      Been Thru The Storm   
417461                                   E I O   
15136                             Empty Spaces   
280793  MYTH  ROID - HYDRA English Translation   
95779                   Ambition / Destruction   
203829              Verb: Thats Whats Happenin   
405924  Everything You Never Had We Had It All   
479959                              We All Lie   
74393                                     Safe   
375663                            Simple Gifts   
27042                                  Hush Bb   
375243           Santa Claus Is Coming To Town   

                                         Artists  \
66730                         Insane Clown Posse   
318324                           Green Carnation   
437090                                      Turk   
340902                                Tupernicus   
378571                    Yung Luciano,DJ Scream   
467784                                2015 Starz   
181199                      Hank Azaria,Ensemble   
88404        DJ Dyce,DJ CANNON BANYON,Yung Ralph   
67143                     J.PERIOD,Black Thought   
417461                           One Acen,Cho8en   
15136                                Linkin Park   
280793                             Manly P. Hall   
95779                               NOTHING MORE   
203829                                     Max Q   
405924                             DJ MixMasters   
479959                                     Hajin   
74393                               EARTHWALKERS   
375663      Christopher D. Azzara,Unknown Artist   
27042   Brian Matthew,Paul McCartney,John Lennon   
375243                              Mariah Carey   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

So as a general rule of thumb, we will drop the songs that have a words_per_sec > 8.

In [ ]:
# Define again the auxiliary columns for measuring singing speed:
# Count number of words in the lyrics
df['word_count'] = df['text'].str.split().apply(len)

# Calculate words per second
df['words_per_sec'] = df['word_count'] / df['Length']

# Inspect songs with suspiciously high values (e.g. above 8 wps)
#suspicious_songs = df3[df3['words_per_sec'] > 8]

In [ ]:
# Use all the space to show the columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Apply the mask
df_short = pd.DataFrame({'indexes': short_songs_df.index})
df_long = pd.DataFrame({'indexes': long_songs_df.index})

In [ ]:
df_extremes = pd.merge(df_short, df_long, on='indexes', how='outer')

In [ ]:
df_short.shape[0], df_long.shape[0], df_extremes.shape[0]

(253, 5, 258)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 450385 entries, 0 to 502193
Data columns (total 43 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Artists                         450385 non-null  object        
 1   song                            450373 non-null  object        
 2   text                            450385 non-null  object        
 3   Length                          450385 non-null  Int64         
 4   emotion                         450385 non-null  object        
 5   Genre                           450385 non-null  object        
 6   Album                           450336 non-null  object        
 7   Release_Date                    450385 non-null  datetime64[ns]
 8   Key                             450385 non-null  object        
 9   Tempo                           450385 non-null  Int64         
 10  Loudness_db                     450385 non-null  float64     

In [ ]:
df_clean2 = df_extremes.merge(
    df,
    left_on='indexes',
    right_index=True,
    how='right'
)

In [ ]:
df.shape[0]

450385

In [ ]:
df_clean = df.drop(df_extremes['indexes'])

In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 450127 entries, 0 to 502193
Data columns (total 43 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Artists                         450127 non-null  object        
 1   song                            450115 non-null  object        
 2   text                            450127 non-null  object        
 3   Length                          450127 non-null  Int64         
 4   emotion                         450127 non-null  object        
 5   Genre                           450127 non-null  object        
 6   Album                           450078 non-null  object        
 7   Release_Date                    450127 non-null  datetime64[ns]
 8   Key                             450127 non-null  object        
 9   Tempo                           450127 non-null  Int64         
 10  Loudness_db                     450127 non-null  float64     

Finally, this is the clean data frame that we will use for our study.

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_clean.to_csv('/content/drive/My Drive/clean_songs_df.csv', index=False)